# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-82b80661c4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    #sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Nobel laureates explorative search") 


Consider the following exploratory scenario:

>  Investigate the authors who won the Nobel award for literature and get information about the nationality of the winners and analyse their literary production



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q37922`   | Nobel Prize in Literature     | node |
| `wd:Q36180`   | writer        | node |
| `wd:Q38`      | Italy         | node |
| `wd:Q3624078` | Sovereign state  | node |
| `wd:Q213678`  | Vatican Library  | node |


Also consider

```
?p wdt:P27 wd:Q38  . 
?p wdt:P106/wdt:P279* wd:Q36180  . 
```

is the BGP to retrieve all **italian authors**

The workload should


1. Identify the BGP for obtaining the nobel laureates and the nominees with their relevant attributes

2. Get the number of Italian, French and Iranian winners and check which nation won more awards in the last twenty years.

3. Determine how many Literature nobel laureates have the following contries: Italy, Germany, France, Romania, Denmark, Iran, and China

4. How many Literature Nobel award winners have a PhD (aka Doctorate of Philosophy) (you may check if they have a doctoral advisor)?

5. Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

In [4]:
#let's see for a nobel in literature if we have properties connecting it to authors
q = f"""
select distinct ?property ?name where{{
?nobel wdt:P31 wd:Q37922 ;
    ?property ?object .
    
?property <http://schema.org/name> ?name
}}
limit 100
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]


2

In [5]:
#then let's start from authors and let's try and see if there is a connection to nobels
q = f"""
select distinct ?property ?name where{{
?author wdt:P27 wd:Q38 ;
    wdt:P106/wdt:P279* wd:Q36180 ;
    ?property wd:Q37922 .
    
?property <http://schema.org/name> ?name
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]


2

In [7]:
#let's retrieve some literature nobel prize winners/nominees
q = f"""
select distinct ?author ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    <http://schema.org/name> ?name ;
    wdt:P1411 | wdt:P166 wd:Q37922
}}
limit 50
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q33760'), ('name', 'Bertrand Russell')]
[('author', 'http://www.wikidata.org/entity/Q37767'), ('name', 'T.\xa0S. Eliot')]
[('author', 'http://www.wikidata.org/entity/Q37030'), ('name', 'Thomas Mann')]
[('author', 'http://www.wikidata.org/entity/Q704931'), ('name', 'Edmund Wilson')]
[('author', 'http://www.wikidata.org/entity/Q309716'), ('name', 'Michel Butor')]
[('author', 'http://www.wikidata.org/entity/Q170509'), ('name', 'Henry James')]
[('author', 'http://www.wikidata.org/entity/Q504'), ('name', 'Émile Zola')]
[('author', 'http://www.wikidata.org/entity/Q235308'), ('name', 'Elizabeth Bowen')]
[('author', 'http://www.wikidata.org/entity/Q38392'), ('name', 'William Faulkner')]
[('author', 'http://www.wikidata.org/entity/Q218679'), ('name', 'Colette')]
[('author', 'http://www.wikidata.org/entity/Q965935'), ('name', 'Henri Queffélec')]
[('author', 'http://www.wikidata.org/entity/Q182804'), ('name', 'Karen Blixen')]
[('author', 'http://www.wik

50

In [8]:
#let's look for interesting attributes
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P1411 | wdt:P166 wd:Q37922;
    ?property ?object .
    
?property <http://schema.org/name> ?name .
}}
limit 100
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('property', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('property', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('property', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('property', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('property', 'http://www.

100

In [9]:
genre = 'wdt:P136'
doctoralAdvisor = 'wdt:P184'
placeOfDetention = 'wdt:P2632'
lifestyle = 'wdt:P1576'
#date of birth/place of birth, gender, spouses are properties that every person usually has. let's look instead into detention and lifestyle
#let's also look for doctorates

q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P1411 | wdt:P166 wd:Q37922;
    ?property ?object .
    
?property <http://schema.org/name> ?name .

filter regex(?name, '.*(doctor|detent|carcer|style).*', 'i')
}}
limit 100
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P1576'), ('name', 'lifestyle')]
[('property', 'http://www.wikidata.org/prop/direct/P184'), ('name', 'doctoral advisor')]
[('property', 'http://www.wikidata.org/prop/direct/P185'), ('name', 'doctoral student')]
[('property', 'http://www.wikidata.org/prop/direct/P2632'), ('name', 'place of detention')]
[('property', 'http://www.wikidata.org/prop/direct/P5459'), ('name', 'RHE doctor ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5415'), ('name', 'Whonamedit? doctor ID')]


6

In [10]:
#we find nothing more

#let's see some authors that were incarcerated and where
q = f"""
select distinct ?name ?placeName where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P1411 | wdt:P166 wd:Q37922;
    <http://schema.org/name> ?name ;
    wdt:P2632 ?place .
    
?place <http://schema.org/name> ?placeName
}}
limit 25
"""
run_query(q)

[('name', 'Bertrand Russell'), ('placeName', 'HM Prison Brixton')]
[('name', 'Paul Celan'), ('placeName', 'Czernowitz Ghetto')]
[('name', 'Aleksandr Solzhenitsyn'), ('placeName', 'Lefortovo Prison')]
[('name', 'Aleksandr Solzhenitsyn'), ('placeName', 'Lubyanka Building')]
[('name', 'Elise Richter'), ('placeName', 'Theresienstadt Ghetto')]
[('name', 'Maxim Gorky'), ('placeName', 'Peter and Paul Fortress')]
[('name', 'Jérôme Carcopino'), ('placeName', 'Fresnes Prison')]
[('name', 'Julien Gracq'), ('placeName', 'Oflag IV-D')]
[('name', 'Imre Kertész'), ('placeName', 'Auschwitz')]
[('name', 'Imre Kertész'), ('placeName', 'Buchenwald concentration camp')]
[('name', 'Arnulf Øverland'), ('placeName', 'Sachsenhausen concentration camp')]


11

In [11]:
#let's see the lifestyles instead
q = f"""
select distinct ?name ?styleName where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P1411 | wdt:P166 wd:Q37922;
    <http://schema.org/name> ?name ;
    wdt:P1576 ?style .
    
?style <http://schema.org/name> ?styleName
}}
limit 25
"""
run_query(q)

[('name', 'Aldous Huxley'), ('styleName', 'mysticism')]
[('name', 'George Bernard Shaw'), ('styleName', 'mysticism')]
[('name', 'T.\xa0S. Eliot'), ('styleName', 'mysticism')]
[('name', 'Nicolai Berdyaev'), ('styleName', 'mysticism')]
[('name', 'Aldous Huxley'), ('styleName', 'vegetarianism')]
[('name', 'George Bernard Shaw'), ('styleName', 'vegetarianism')]
[('name', 'Shmuel Yosef Agnon'), ('styleName', 'vegetarianism')]
[('name', 'Isaac Bashevis Singer'), ('styleName', 'vegetarianism')]


8

In [12]:
#not much information to be found there

#what about genre?
q = f"""
select distinct ?name ?genreName where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P1411 | wdt:P166 wd:Q37922;
    <http://schema.org/name> ?name ;
    wdt:P136 ?genre .
    
?genre <http://schema.org/name> ?genreName
}}
limit 25
"""
run_query(q)

[('name', 'T.\xa0S. Eliot'), ('genreName', 'fiction literature')]
[('name', 'T.\xa0S. Eliot'), ('genreName', 'poetry')]
[('name', 'Karen Blixen'), ('genreName', 'neo-romanticism')]
[('name', 'Émile Zola'), ('genreName', 'novel')]
[('name', 'Henry James'), ('genreName', 'Gothic literature')]
[('name', 'Karen Blixen'), ('genreName', 'Gothic literature')]
[('name', 'Émile Zola'), ('genreName', 'short story')]
[('name', 'Karen Blixen'), ('genreName', 'magic realism')]
[('name', 'Nadine Gordimer'), ('genreName', 'novel')]
[('name', 'Najib Mahfouz'), ('genreName', 'novel')]
[('name', 'Gabriel García Márquez'), ('genreName', 'novel')]
[('name', 'Nadine Gordimer'), ('genreName', 'short story')]
[('name', 'Alice Munro'), ('genreName', 'short story')]
[('name', 'Najib Mahfouz'), ('genreName', 'short story')]
[('name', 'Nadine Gordimer'), ('genreName', 'novella')]
[('name', 'José Saramago'), ('genreName', 'novel')]
[('name', 'José Saramago'), ('genreName', 'magic realist fiction')]
[('name', 'Dar

25

In [13]:
#let's count the number of genres across nobel winners authors
q = f"""
select ?genre ?genreName (count(?author) as ?howmany) where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P166 wd:Q37922;
    wdt:P136 ?genre .
    
?genre <http://schema.org/name> ?genreName
}}
group by ?genre ?genreName
order by desc(?howmany)
"""
run_query(q)

[('genre', 'http://www.wikidata.org/entity/Q8261'), ('genreName', 'novel'), ('howmany', '145')]
[('genre', 'http://www.wikidata.org/entity/Q482'), ('genreName', 'poetry'), ('howmany', '73')]
[('genre', 'http://www.wikidata.org/entity/Q49084'), ('genreName', 'short story'), ('howmany', '52')]
[('genre', 'http://www.wikidata.org/entity/Q676'), ('genreName', 'prose'), ('howmany', '38')]
[('genre', 'http://www.wikidata.org/entity/Q35760'), ('genreName', 'essay'), ('howmany', '35')]
[('genre', 'http://www.wikidata.org/entity/Q492537'), ('genreName', 'dramaturgy'), ('howmany', '20')]
[('genre', 'http://www.wikidata.org/entity/Q11635'), ('genreName', 'theatre'), ('howmany', '15')]
[('genre', 'http://www.wikidata.org/entity/Q156035'), ('genreName', 'opinion journalism'), ('howmany', '14')]
[('genre', 'http://www.wikidata.org/entity/Q102439494'), ('genreName', 'magic realist fiction'), ('howmany', '14')]
[('genre', 'http://www.wikidata.org/entity/Q38072107'), ('genreName', 'fiction literature')

43

In [14]:
#since we didn't get very interesting information when looking for lifestyles, place of detention and so on
#let's go back and find more general properties

q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P1411 | wdt:P166 wd:Q37922;
    ?property ?object .
    
?property <http://schema.org/name> ?name .
filter regex (?name, '.*(age|spouse|children|sibling|brother|sister|birth|death|gender|husband|wife).*')
}}
limit 100
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('property', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('property', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('property', 'http://www.wikidata.org/prop/direct/P1442'), ('name', 'image of grave')]
[('property', 'http://www.wikidata.org/prop/direct/P1472'), ('name', 'Commons Creator page')]
[('property', 'http://www.wikidata.org/prop/direct/P1477'), ('name', 'birth name')]
[('property', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('property', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('property', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('property', 'ht

40

In [15]:
mannerOfDeath = 'wdt:P1196'
gender = 'wdt:P21'
spouse = 'wdt:P26'
sibling = 'wdt:P3373'
deathDate = 'wdt:P570'
birthDate = 'wdt:P569'
nChildren = 'wdt:P1971'

#let's count the number of spuses each nobel prize winner had
q = f"""
select ?author ?authorName (count(?spouse) as ?nSpouses) where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    <http://schema.org/name> ?authorName ;
    wdt:P166 wd:Q37922;
    wdt:P26 ?spouse .
    
}}
group by ?author ?authorName
order by desc(?nSpouses)
limit 100
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q33760'), ('authorName', 'Bertrand Russell'), ('nSpouses', '72')]
[('author', 'http://www.wikidata.org/entity/Q37060'), ('authorName', 'José Saramago'), ('nSpouses', '42')]
[('author', 'http://www.wikidata.org/entity/Q37767'), ('authorName', 'T.\xa0S. Eliot'), ('nSpouses', '26')]
[('author', 'http://www.wikidata.org/entity/Q23434'), ('authorName', 'Ernest Hemingway'), ('nSpouses', '24')]
[('author', 'http://www.wikidata.org/entity/Q39803'), ('authorName', 'Mario Vargas Llosa'), ('nSpouses', '24')]
[('author', 'http://www.wikidata.org/entity/Q25351'), ('authorName', 'Theodor Mommsen'), ('nSpouses', '22')]
[('author', 'http://www.wikidata.org/entity/Q34670'), ('authorName', 'Antonio carglass'), ('nSpouses', '20')]
[('author', 'http://www.wikidata.org/entity/Q47619'), ('authorName', 'Nadine Gordimer'), ('nSpouses', '16')]
[('author', 'http://www.wikidata.org/entity/Q34474'), ('authorName', 'Aleksandr Solzhenitsyn'), ('nSpouses', '16')]
[('author

69

In [16]:
#that can't be right, let's use distinct
q = f"""
select ?author ?authorName (count(distinct ?spouse) as ?nSpouses) where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    <http://schema.org/name> ?authorName ;
    wdt:P166 wd:Q37922;
    wdt:P26 ?spouse .
    
}}
group by ?author ?authorName
order by desc(?nSpouses)
limit 100
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q33760'), ('authorName', 'Bertrand Russell'), ('nSpouses', '4')]
[('author', 'http://www.wikidata.org/entity/Q23434'), ('authorName', 'Ernest Hemingway'), ('nSpouses', '4')]
[('author', 'http://www.wikidata.org/entity/Q41502'), ('authorName', 'Henryk Sienkiewicz'), ('nSpouses', '3')]
[('author', 'http://www.wikidata.org/entity/Q25973'), ('authorName', 'Hermann Hesse'), ('nSpouses', '3')]
[('author', 'http://www.wikidata.org/entity/Q39212'), ('authorName', 'John Steinbeck'), ('nSpouses', '3')]
[('author', 'http://www.wikidata.org/entity/Q37060'), ('authorName', 'José Saramago'), ('nSpouses', '3')]
[('author', 'http://www.wikidata.org/entity/Q131326'), ('authorName', 'Eyvind Johnson'), ('nSpouses', '2')]
[('author', 'http://www.wikidata.org/entity/Q47755'), ('authorName', 'Imre Kertész'), ('nSpouses', '2')]
[('author', 'http://www.wikidata.org/entity/Q41223'), ('authorName', 'Boris Pasternak'), ('nSpouses', '2')]
[('author', 'http://www.wikidat

69

In [17]:
#now let's use a subquery

subQ = f"""
select distinct ?author ?spouse where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P166 wd:Q37922 ;
    wdt:P26 ?spouse .
}}
"""

q = f"""
select ?author ?authorName (count(?author) as ?nSpouses) where{{
        {{
            select distinct ?author ?spouse where{{

?author wdt:P106/wdt:P279* wd:Q36180;

    wdt:P166 wd:Q37922 ;

    wdt:P26 ?spouse .

}}



        }}
?author <http://schema.org/name> ?authorName .

}}
group by ?author ?authorName
order by desc(?nSpouses)
limit 100
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q23434'), ('authorName', 'Ernest Hemingway'), ('nSpouses', '4')]
[('author', 'http://www.wikidata.org/entity/Q33760'), ('authorName', 'Bertrand Russell'), ('nSpouses', '4')]
[('author', 'http://www.wikidata.org/entity/Q37060'), ('authorName', 'José Saramago'), ('nSpouses', '3')]
[('author', 'http://www.wikidata.org/entity/Q39212'), ('authorName', 'John Steinbeck'), ('nSpouses', '3')]
[('author', 'http://www.wikidata.org/entity/Q41502'), ('authorName', 'Henryk Sienkiewicz'), ('nSpouses', '3')]
[('author', 'http://www.wikidata.org/entity/Q25973'), ('authorName', 'Hermann Hesse'), ('nSpouses', '3')]
[('author', 'http://www.wikidata.org/entity/Q38049'), ('authorName', 'Herta Müller'), ('nSpouses', '2')]
[('author', 'http://www.wikidata.org/entity/Q37767'), ('authorName', 'T.\xa0S. Eliot'), ('nSpouses', '2')]
[('author', 'http://www.wikidata.org/entity/Q46602'), ('authorName', 'Ivan Bunin'), ('nSpouses', '2')]
[('author', 'http://www.wikidata.org/

69

In [18]:
#let's see them ordered by longevity
q = f"""
select ?author ?authorName ?age where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    <http://schema.org/name> ?authorName ;
    wdt:P166 wd:Q37922;
    wdt:P569 ?birthDate ;
    wdt:P570 ?deathDate .
    
bind (year(?deathDate) - year(?birthDate) as ?age) .
    
}}
group by ?author ?authorName
order by desc(?age)
limit 100
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q33760'), ('authorName', 'Bertrand Russell'), ('age', '98')]
[('author', 'http://www.wikidata.org/entity/Q80321'), ('authorName', 'Halldór Laxness'), ('age', '96')]
[('author', 'http://www.wikidata.org/entity/Q7176'), ('authorName', 'Najib Mahfouz'), ('age', '95')]
[('author', 'http://www.wikidata.org/entity/Q40874'), ('authorName', 'Doris Lessing'), ('age', '94')]
[('author', 'http://www.wikidata.org/entity/Q19185'), ('authorName', 'George Bernard Shaw'), ('age', '94')]
[('author', 'http://www.wikidata.org/entity/Q45970'), ('authorName', 'Czesław Miłosz'), ('age', '93')]
[('author', 'http://www.wikidata.org/entity/Q40826'), ('authorName', 'Knut Hamsun'), ('age', '93')]
[('author', 'http://www.wikidata.org/entity/Q131549'), ('authorName', 'Claude Simon'), ('age', '92')]
[('author', 'http://www.wikidata.org/entity/Q8016'), ('authorName', 'Winston Churchill'), ('age', '91')]
[('author', 'http://www.wikidata.org/entity/Q47619'), ('authorName', '

100

In [19]:
#that formula doesn't consider months or days, google says bertrand russel died at 97, not 98
#that's still a calculation i would do on the front end if need be, especially since there doesn't seem to be an age() function

#last query for the relevant attributes of an author: let's group them by the manner of death

q = f"""
select ?mannerOfDeath ?name (count(distinct ?author) as ?howmany) where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P166 wd:Q37922;
    wdt:P1196 ?mannerOfDeath .
    
?mannerOfDeath <http://schema.org/name> ?name .
    
}}
group by ?mannerOfDeath ?name
order by desc(?howmany)
limit 100
"""
run_query(q)

[('mannerOfDeath', 'http://www.wikidata.org/entity/Q3739104'), ('name', 'natural causes'), ('howmany', '27')]
[('mannerOfDeath', 'http://www.wikidata.org/entity/Q10737'), ('name', 'suicide'), ('howmany', '2')]
[('mannerOfDeath', 'http://www.wikidata.org/entity/Q171558'), ('name', 'accident'), ('howmany', '2')]
[('mannerOfDeath', 'http://www.wikidata.org/entity/Q133462'), ('name', 'seppuku'), ('howmany', '1')]


4

In [20]:
#2. Get the number of Italian, French and Iranian winners and check which nation won more awards in the last twenty years.

#first of all let's get the nodes for Italy, France, Iran

q = f"""
select distinct ?country ?countryName where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P166 wd:Q37922;
    wdt:P17 ?country .
    
?country <http://schema.org/name> ?countryName .

filter regex(?countryName, '.*(italy|iran|france).*', 'i')

}}
"""
run_query(q)

Empty


0

In [21]:
#let's see if there is another property for country 
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P166 wd:Q37922;
    ?property ?object .
    
?property <http://schema.org/name> ?name .

filter regex(?name, 'country', 'i')

}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]


1

In [22]:
citizenship = 'wdt:P27'
#let's see the country of each author
q = f"""
select distinct ?author ?authorName ?country ?countryName where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P166 wd:Q37922;
    <http://schema.org/name> ?authorName ;
    wdt:P27 ?country .
    
?country <http://schema.org/name> ?countryName

}}
order by asc(?countryName)
limit 100
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q43293'), ('authorName', 'J. M. Coetzee'), ('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('author', 'http://www.wikidata.org/entity/Q129187'), ('authorName', 'Patrick White'), ('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('author', 'http://www.wikidata.org/entity/Q47243'), ('authorName', 'Elfriede Jelinek'), ('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria')]
[('author', 'http://www.wikidata.org/entity/Q44107'), ('authorName', 'Peter Handke'), ('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria')]
[('author', 'http://www.wikidata.org/entity/Q41502'), ('authorName', 'Henryk Sienkiewicz'), ('country', 'http://www.wikidata.org/entity/Q28513'), ('countryName', 'Austria-Hungary')]
[('author', 'http://www.wikidata.org/entity/Q133042'), ('authorName', 'Shmuel Yosef Agnon'), ('country', 'http://www.wikidata.org/entity/Q28513')

100

In [23]:
#let's try again
q = f"""
select distinct ?country ?countryName where{{
?author wdt:P106/wdt:P279* wd:Q36180;
    wdt:P166 wd:Q37922;
    wdt:P27 ?country .
    
?country <http://schema.org/name> ?countryName .

filter regex(?countryName, '.*(italy|iran|france).*', 'i') .

}}
order by asc(?countryName)
limit 100
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q172579'), ('countryName', 'Kingdom of Italy')]


3

In [24]:
#no iran...
#let's just see the winners for italy, france, iran

q = f"""
select distinct ?author ?authorName ?countryName where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?authorName ;
    wdt:P27 ?country .
    
?country <http://schema.org/name> ?countryName .

filter regex(?countryName, '(italy|iran|france)', 'i')

}}
order by asc(?countryName)
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q9364'), ('authorName', 'Jean-Paul Sartre'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q34670'), ('authorName', 'Antonio carglass'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q47484'), ('authorName', 'André Gide'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q42037'), ('authorName', 'Jean-Marie Gustave Le Clézio'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q47162'), ('authorName', 'Romain Rolland'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q42247'), ('authorName', 'Sully Prudhomme'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q42443'), ('authorName', 'Anatole France'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q46602'), ('authorName', 'Ivan Bunin'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q42156'), ('authorName', '

25

In [25]:
#to retrieve the date the award was received i need more prefixes though...
p_awarded = 'p:P166'

#let's retrieve qualifiers for awarded
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    p:P166 ?awardedProperty .
    
?awardedProperty ?property ?value .

?property <http://schema.org/name> ?name .
    
}}
"""
run_query(q)

The operation failed QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 13: Undefined namespace prefix at 'p' before '?awardedProperty'\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-82b80661c4-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nselect distinct ?property ?name where{\n?author wdt:P106/wdt:P279* wd:Q36180 ;\n    wdt:P166 wd:Q37922 ;\n    p:P166 ?awardedProperty .\n    \n?awardedProperty ?property ?value .\n\n?property <http://schema.org/name> ?name .\n    \n}\n"


In [26]:
#let's try with the url
p_awarded = '<http://www.wikidata.org/prop/P166>'
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    <http://www.wikidata.org/prop/P166> ?awardedProperty .
    
?awardedProperty ?property ?value .

?property <http://schema.org/name> ?name .
    
}}
"""
run_query(q)

Empty


0

In [27]:
#let's remove awarded
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    <http://www.wikidata.org/prop/P166> ?awardedProperty .
    
?awardedProperty ?property ?value .

?property <http://schema.org/name> ?name .
    
}}
"""
run_query(q)

Empty


0

In [28]:
#let's try and remove the name
q = f"""
select distinct ?property where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    <http://www.wikidata.org/prop/P166> ?awardedProperty .
    
?awardedProperty ?property ?value .
    
}}
"""
run_query(q)

Empty


0

In [29]:
#the only thing i can think of is that there may be an entry of the type "Nobel prize XXXX" where XXXX is the year
#i know that formula 1 seasons are stored like that

#let's look at awards received by writers, doing a regex on the name of the award, checking if some have a year in the name
q = f"""
select distinct ?award ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 ?prize .
    
?prize <http://schema.org/name> ?name .

filter regex(?name, '.*(nobel).*', 'i')
}}
limit 100
"""
run_query(q)

[('name', 'Ig Nobel Prize')]
[('name', 'Nobel Peace Prize')]
[('name', 'Nobel Prize in Literature')]
[('name', 'Nobel Prize in Physics')]
[('name', 'Nobel Prize in Physiology or Medicine')]
[('name', 'Nobel Prize in Chemistry')]
[('name', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('name', 'list of Nobel laureates in Physiology or Medicine')]
[('name', 'Nobel Laureate Meetings at Lindau')]


9

In [30]:
#the (incomplete) answer to point 2 is then
q = f"""
select distinct ?author ?authorName ?countryName where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?authorName ;
    wdt:P27 ?country .
    
?country <http://schema.org/name> ?countryName .

filter regex(?countryName, '(italy|iran|france)', 'i')

}}
order by asc(?countryName)
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q9364'), ('authorName', 'Jean-Paul Sartre'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q34670'), ('authorName', 'Antonio carglass'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q47484'), ('authorName', 'André Gide'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q42037'), ('authorName', 'Jean-Marie Gustave Le Clézio'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q47162'), ('authorName', 'Romain Rolland'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q42247'), ('authorName', 'Sully Prudhomme'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q42443'), ('authorName', 'Anatole France'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q46602'), ('authorName', 'Ivan Bunin'), ('countryName', 'France')]
[('author', 'http://www.wikidata.org/entity/Q42156'), ('authorName', '

25

In [31]:
#3. Determine how many Literature nobel laureates have the following contries: Italy, Germany, France, Romania, Denmark, Iran, and China
q = f"""
select ?country ?countryName (count(distinct ?author) as ?howmany) where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P27 ?country .
    
?country <http://schema.org/name> ?countryName .

filter regex(?countryName, '(italy|iran|france|germany|romania|denmark|iran|china)', 'i')

}}
group by ?country ?countryName
order by desc(?howmany)
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('howmany', '17')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('howmany', '6')]
[('country', 'http://www.wikidata.org/entity/Q172579'), ('countryName', 'Kingdom of Italy'), ('howmany', '5')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('howmany', '4')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('howmany', '3')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryName', "People's Republic of China"), ('howmany', '2')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('howmany', '1')]
[('country', 'http://www.wikidata.org/entity/Q7318'), ('countryName', 'Nazi Germany'), ('howmany', '1')]


8

In [32]:
#let's see the french ones, on wikipedia i counted only 16
q = f"""
select distinct ?author ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?name ;
    wdt:P27 ?country .
    
?country <http://schema.org/name> ?countryName .

filter regex(?countryName, 'france', 'i')

}}
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q9364'), ('name', 'Jean-Paul Sartre')]
[('author', 'http://www.wikidata.org/entity/Q34670'), ('name', 'Antonio carglass')]
[('author', 'http://www.wikidata.org/entity/Q47484'), ('name', 'André Gide')]
[('author', 'http://www.wikidata.org/entity/Q42037'), ('name', 'Jean-Marie Gustave Le Clézio')]
[('author', 'http://www.wikidata.org/entity/Q47162'), ('name', 'Romain Rolland')]
[('author', 'http://www.wikidata.org/entity/Q42247'), ('name', 'Sully Prudhomme')]
[('author', 'http://www.wikidata.org/entity/Q42443'), ('name', 'Anatole France')]
[('author', 'http://www.wikidata.org/entity/Q46602'), ('name', 'Ivan Bunin')]
[('author', 'http://www.wikidata.org/entity/Q42156'), ('name', 'Henri Bergson')]
[('author', 'http://www.wikidata.org/entity/Q129155'), ('name', 'Roger Martin du Gard')]
[('author', 'http://www.wikidata.org/entity/Q37327'), ('name', 'Samuel Beckett')]
[('author', 'http://www.wikidata.org/entity/Q81685'), ('name', 'François Mauriac')

17

In [34]:
#4. How many Literature Nobel award winners have a PhD (aka Doctorate of Philosophy) (you may check if they have a doctoral advisor)?
#let's retrieve them all
q = f"""
select distinct ?author ?authorName where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?authorName .
    
filter exists {{
            ?author wdt:P184 ?advisor
        }}
}}
limit 100
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q33760'), ('authorName', 'Bertrand Russell')]
[('author', 'http://www.wikidata.org/entity/Q47695'), ('authorName', 'Rudolf Christoph Eucken')]


2

In [35]:
#there's not a lot of them... let's rewrite the query
q = f"""
select distinct ?author ?authorName where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?authorName ;
    wdt:P184 ?advisor .

}}
limit 100
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q33760'), ('authorName', 'Bertrand Russell')]
[('author', 'http://www.wikidata.org/entity/Q47695'), ('authorName', 'Rudolf Christoph Eucken')]


2

In [36]:
#is there a relationship related to instruction?
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?authorName ;
    ?property ?value .
    
?property <http://schema.org/name> ?name .

filter regex(?property, '.*(doctor|doc|instru|phd|stud|title|philoso).*', 'i')
}}
limit 100
"""
run_query(q)

Empty


0

In [37]:
#5. Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

#first of all we need to find out the relationship for the works of an author.
#let's take bertrand russel. a quick search tells us he wrote "The Problems of Philosophy". we use that to look for the relevant relationship

BertrandRussel = 'wd:Q33760'

q = f"""
select distinct ?property ?name where{{
wd:Q33760 ?property ?value .
?property <http://schema.org/name> ?name .
?value <http://schema.org/name> ?vName .
filter regex(?vName, '.*problems of philosophy.*', 'i') .
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')]


1

In [38]:
notableWork = 'wdt:P800'
#now we have to figure out the property connecting a book to the vatican library

q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book ?property ?object .
?property <http://schema.org/name> ?name .

?object <http://schema.org/name> ?objectName .
filter regex(?objectName, '.*vatican.*', 'i') .
}}
"""
run_query(q)

Empty


0

In [39]:
#instead let's look at the properties for books
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book ?property ?object .
?property <http://schema.org/name> ?name .

?object <http://schema.org/name> ?objectName .
}}
limit 100
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P6962'), ('name', 'narrative motif')]
[('property', 'http://www.wikidata.org/prop/direct/P8371'), ('name', 'references work, tradition or theory')]
[('property', 'http://www.wikidata.org/prop/direct/P123'), ('name', 'publisher')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('property', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('property', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P291'), ('name', 'place of publication')]
[('property', 'http://www.wikidata.org/prop/dire

82

In [40]:
#no property seems to indicate an archive
#is it possible that the inverse is true? that it's the vatican archive to hold a certain book?

q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .

?vatican ?property ?book .
?vatican <http://schema.org/name> ?vaticanName .
?property <http://schema.org/name> ?name .

filter regex (?vaticanName, '.*vatican.*', ',') .
}}
limit 100
"""
run_query(q)

Empty


0

In [41]:
#maybe the archive is on the authors themselves?
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    ?property ?vatican .

?property <http://schema.org/name> ?name .

?vatican <http://schema.org/name> ?vaticanName .
filter regex(?vaticanName, '.*vatican.*', 'i') .
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')]


1

In [42]:
#let's try and find the vatican library by a roundabout way, first by finding the vatican country, then by finding everything related to that country

#first we have to find the class for a country, let's use italy

q = f"""
select distinct ?class ?name where{{
wd:Q38 wdt:P31 ?class .
?class <http://schema.org/name> ?name
}}
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('class', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]


2

In [43]:
Country = 'wd:Q6256'
q = f"""
select distinct ?country ?name where{{
?country wdt:P31 wd:Q6256 ;
    <http://schema.org/name> ?name .
    
filter regex(?name, '.*vatican.*', 'i')
}}
"""
run_query(q)

Empty


0

In [44]:
q = f"""
select distinct ?country ?name where{{
?country wdt:P31 wd:Q6256 ;
    <http://schema.org/name> ?name .
    
filter regex(?name, 'vatican city', 'i')
}}
"""
run_query(q)

Empty


0

In [45]:
q = f"""
select distinct ?country ?name where{{
?country wdt:P31 wd:Q6256 ;
    <http://schema.org/name> ?name .
    
filter regex(?name, '.*del vaticano', 'i')
}}
"""
run_query(q)

Empty


0

In [46]:
#let's try with sovereign state
SovereignState = 'wd:Q3624078'
q = f"""
select distinct ?country ?name where{{
?country wdt:P31 wd:Q3624078 ;
    <http://schema.org/name> ?name .
    
filter regex(?name, '.*vatican.*', 'i')
}}
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]


1

In [47]:
Vatican = 'wd:Q237'
#now let's try and see everything with "library" or "archive" that connects with the vatican city via the "country" property
q = f"""
select distinct ?subject ?name where{{
?subject wdt:P17 wd:Q237 ;
    <http://schema.org/name> ?name .
    
filter regex(?name, '.*(library|archive|libr|arch).*', 'i')
}}
"""
run_query(q)

[('subject', 'http://www.wikidata.org/entity/Q536580'), ('name', 'Vatican Apostolic Archive')]
[('subject', 'http://www.wikidata.org/entity/Q213678'), ('name', 'Vatican Library')]
[('subject', 'http://www.wikidata.org/entity/Q105848309'), ('name', 'Archpriest of the Basilica di San Pietro in Vaticano')]
[('subject', 'http://www.wikidata.org/entity/Q19823475'), ('name', 'prefect of the Vatican Library')]
[('subject', 'http://www.wikidata.org/entity/Q12051107'), ('name', 'Scuola vaticana di paleografia, diplomatica e archivistica')]
[('subject', 'http://www.wikidata.org/entity/Q104093644'), ('name', 'Archivist of the Vatican Archive')]
[('subject', 'http://www.wikidata.org/entity/Q104094089'), ('name', 'Prefect of the Vatican Archives')]
[('subject', 'http://www.wikidata.org/entity/Q3745476'), ('name', 'Vatican Film Library')]
[('subject', 'http://www.wikidata.org/entity/P8034'), ('name', 'Vatican Library VcBA ID')]
[('subject', 'http://www.wikidata.org/entity/Q50421021'), ('name', 'The 

18

In [48]:
#we found the Vatican Library entity
VaticanLibrary = 'wd:Q213678'

#let's attempt to connect works of nobel laureates in literature with the vatican library
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book ?property wd:Q213678 .
    
?property <http://schema.org/name> ?name
}}
"""
run_query(q)

Empty


0

In [49]:
#let's try with 2 jumps
q = f"""
select distinct ?property1 ?property2 ?name1 ?name2 where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book ?property1 ?middle .
?middle ?property2 wd:Q213678 .
    
?property1 <http://schema.org/name> ?name1 .
?property2 <http://schema.org/name> ?name2 .
}}
"""
run_query(q)

Empty


0

In [50]:
#let's attempt and connect nobel laureates to the vatican library directly
q = f"""
select distinct ?property1 ?name1 where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 .
    
?author ?property1 wd:Q213678 .
    
?property1 <http://schema.org/name> ?name1 .

}}
"""
run_query(q)

Empty


0

In [51]:
#let's attempt and connect nobel laureates to the vatican library directly
q = f"""
select distinct ?property1 ?name1 ?property2 ?name2 where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 .
    
?author ?property1 ?middle .
?middle ?property2 wd:Q213678 .
    
?property1 <http://schema.org/name> ?name1 .
?property2 <http://schema.org/name> ?name2 .

}}
"""
run_query(q)

Empty


0

In [52]:
#maybe this is the wrong approach. is there a property for a book, related to vatican library, not pointing to it?
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book ?property ?value .
?property <http://schema.org/name> ?name .

filter regex(?name, '.*vatican.*', 'i') .

}}
"""
run_query(q)

Empty


0

In [53]:
#let's see all properties exiting notable works
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book ?property ?value .
?property <http://schema.org/name> ?name .

}}
limit 100
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P6962'), ('name', 'narrative motif')]
[('property', 'http://www.wikidata.org/prop/direct/P8371'), ('name', 'references work, tradition or theory')]
[('property', 'http://www.wikidata.org/prop/direct/P1844'), ('name', 'HathiTrust ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2093'), ('name', 'author name string')]
[('property', 'http://www.wikidata.org/prop/direct/P1085'), ('name', 'LibraryThing work ID')]
[('property', 'http://www.wikidata.org/prop/direct/P123'), ('name', 'publisher')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P1375'), ('name', 'NSK ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('property', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('property', 'http:/

100

In [54]:
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book ?property ?value .
?property <http://schema.org/name> ?name .

}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P6962'), ('name', 'narrative motif')]
[('property', 'http://www.wikidata.org/prop/direct/P8371'), ('name', 'references work, tradition or theory')]
[('property', 'http://www.wikidata.org/prop/direct/P1844'), ('name', 'HathiTrust ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2093'), ('name', 'author name string')]
[('property', 'http://www.wikidata.org/prop/direct/P1085'), ('name', 'LibraryThing work ID')]
[('property', 'http://www.wikidata.org/prop/direct/P123'), ('name', 'publisher')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P1375'), ('name', 'NSK ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('property', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('property', 'http:/

264

In [55]:
#this idea didn't seem bad. there are many IDs related to different collections, but not a single property for a vatican library id
#let's try and see if the laureates instead have some property like that
q = f"""
select distinct ?property ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    ?property ?value .
    
?property <http://schema.org/name> ?name .

filter regex(?name, '.*vatican.*', 'i')

}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P8034'), ('name', 'Vatican Library VcBA ID')]


2

In [56]:
#we have some authors with a vatican library id.
#since we're missing (or we can't find) connections to the vatican library for the single books, to answer question 5 we will simply
#count how many books each author has, when an author is not related to the vatican library

vaticanLibraryId = 'wdt:P1017'
vaticanLibraryVcba = 'wdt:P8034'

#retrieve authors without any association to the vatican library
q1 = f"""
select distinct ?author where {{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 .
    
    filter not exists {{
        ?author wdt:P1017 | wdt:P8034 ?id
    }}  
}}
"""

q = f"""
select ?author ?name (count(?book) as ?n) where{{
    ?author wdt:P800 ?book ;
        <http://schema.org/name> ?name .
        
    {{
        select distinct ?author where {{

?author wdt:P106/wdt:P279* wd:Q36180 ;

    wdt:P166 wd:Q37922 .

    

    filter not exists {{

        ?author wdt:P1017 | wdt:P8034 ?id

    }}  

}}



    }}
}}
group by ?author ?name
order by desc(?n)
limit 10
"""
run_query(q)

[('author', 'http://www.wikidata.org/entity/Q7176'), ('name', 'Najib Mahfouz'), ('n', '24')]
[('author', 'http://www.wikidata.org/entity/Q72334'), ('name', 'Toni Morrison'), ('n', '11')]
[('author', 'http://www.wikidata.org/entity/Q254032'), ('name', 'Olga Tokarczuk'), ('n', '10')]
[('author', 'http://www.wikidata.org/entity/Q392'), ('name', 'Bob Dylan'), ('n', '7')]
[('author', 'http://www.wikidata.org/entity/Q241248'), ('name', 'Orhan Pamuk'), ('n', '7')]
[('author', 'http://www.wikidata.org/entity/Q133042'), ('name', 'Shmuel Yosef Agnon'), ('n', '7')]
[('author', 'http://www.wikidata.org/entity/Q47619'), ('name', 'Nadine Gordimer'), ('n', '6')]
[('author', 'http://www.wikidata.org/entity/Q40874'), ('name', 'Doris Lessing'), ('n', '6')]
[('author', 'http://www.wikidata.org/entity/Q44107'), ('name', 'Peter Handke'), ('n', '6')]
[('author', 'http://www.wikidata.org/entity/Q43736'), ('name', 'Yasunari Kawabata'), ('n', '6')]


10

In [58]:
#coming back to the properties exiting a book, there is a "collection" property which has my interest
collection = 'wdt:P195'
q = f"""
select distinct ?collection ?name where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book wdt:P195 ?collection .
?collection <http://schema.org/name> ?name .

}}
"""
run_query(q)

[('collection', 'http://www.wikidata.org/entity/Q1895953'), ('name', 'National museum of modern art')]


1

In [59]:
#let's attempt the inverse, connect the vatican library to a book
q = f"""
select distinct ?property1 ?name1 where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
wd:Q213678 ?property1 ?book .
?property1 <http://schema.org/name> ?name1 .

}}
"""
run_query(q)

Empty


0

In [60]:
q = f"""
select distinct ?property1 ?name1 ?property2 ?name2 where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
wd:Q213678 ?property1 ?middle .
?middle ?property2 ?book .
?property1 <http://schema.org/name> ?name1 .
?property2 <http://schema.org/name> ?name2 .

}}
"""
run_query(q)

Empty


0

In [61]:
q = f"""
select distinct ?property1 ?name1 ?property2 ?name2 ?property3 ?name3 where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
wd:Q213678 ?property1 ?middle1 .
?middle1 ?property2 ?middle2 .
?middle2 ?property3 ?book .
?property1 <http://schema.org/name> ?name1 .
?property2 <http://schema.org/name> ?name2 .
?property3 <http://schema.org/name> ?name3 .

}}
"""
run_query(q)

Empty


0

In [62]:
#i googled to find famous books stored in the vatican library, one of them is Codex Vaticanus
#let's try and find its entity, if it exists

#since we don't know the class for a book yet, nor we know if Codex Vaticanus has that class, we filter regex by its name, also connecting it already to the vatican library
#we then look for the property that connects it to the vatican library

q = f"""
select distinct ?book ?name ?property ?propertyName where{{
?book <http://schema.org/name> ?name ;
    ?property wd:Q213678 .
    
?property <http://schema.org/name> ?propertyName .

filter regex(?name, '.*codex.*vaticanus.*', 'i') .
}}
"""
run_query(q)

[('book', 'http://www.wikidata.org/entity/Q55831099'), ('name', 'Codex vaticanus latinus 3199'), ('property', 'http://www.wikidata.org/prop/direct/P195'), ('propertyName', 'collection')]
[('book', 'http://www.wikidata.org/entity/Q1106341'), ('name', 'Codex Vaticanus Graecus 64'), ('property', 'http://www.wikidata.org/prop/direct/P195'), ('propertyName', 'collection')]
[('book', 'http://www.wikidata.org/entity/Q1106357'), ('name', 'Codex Vaticanus 2061'), ('property', 'http://www.wikidata.org/prop/direct/P195'), ('propertyName', 'collection')]
[('book', 'http://www.wikidata.org/entity/Q209285'), ('name', 'Codex Vaticanus'), ('property', 'http://www.wikidata.org/prop/direct/P195'), ('propertyName', 'collection')]
[('book', 'http://www.wikidata.org/entity/Q5140259'), ('name', 'Codex Vaticanus Latinus 3868'), ('property', 'http://www.wikidata.org/prop/direct/P195'), ('propertyName', 'collection')]
[('book', 'http://www.wikidata.org/entity/Q1106358'), ('name', 'Codex Vaticanus 2066'), ('pro

9

In [63]:
#now we are sure that either collection or location connect a book to the vatican library
location = 'wdt:P276'

#let's go back to the books of nobel laureates and see if they are located in the vatican library
q = f"""
select distinct ?author ?authorName ?book ?bookName where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?authorName ;
    wdt:P800 ?book .
    
?book wdt:P195 | wdt:P276 wd:Q213678;
    <http://schema.org/name> ?bookName

}}
limit 20
"""
run_query(q)

Empty


0

In [64]:
#just to make sure, let's see if there are authors, not nobel laureates, with books stored in the vatican library
q = f"""
select distinct ?author ?authorName ?book ?bookName where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    <http://schema.org/name> ?authorName ;
    wdt:P800 ?book .
    
?book wdt:P195 | wdt:P276 wd:Q213678;
    <http://schema.org/name> ?bookName

}}
limit 20
"""
run_query(q)

Empty


0

In [65]:
q = f"""
select distinct ?property ?propertyName ?book ?bookName where{{
?author wdt:P106/wdt:P279* wd:Q36180 ;
    <http://schema.org/name> ?authorName ;
    wdt:P800 ?book .
    
?book ?property wd:Q213678;
    <http://schema.org/name> ?bookName .
    
?property <http://schema.org/name> ?propertyName

}}
limit 20
"""
run_query(q)

Empty


0

In [66]:
#i'm out of ideas at this point, let's see if we remove the author
q = f"""
select distinct ?bookName where{{
    
?book wdt:P195|wdt:P276 wd:Q213678;
    <http://schema.org/name> ?bookName .
    
}}
limit 20
"""
run_query(q)

[('bookName', 'Bibbia di Federico da Montefeltro - BAV Urb.Lat.1&2')]
[('bookName', 'Codex Vaticanus')]
[('bookName', 'Lasimos Krater')]
[('bookName', 'Bodmer Papyri')]
[('bookName', 'Codex Vaticanus Graecus 64')]
[('bookName', 'Barberini Gospels')]
[('bookName', 'Divina Commedia de Federigo da Montefeltro - BAVaticana UrbLat365')]
[('bookName', 'Book of Dede Korkut')]
[('bookName', 'Rossiana Library')]
[('bookName', 'Vergilius Vaticanus')]
[('bookName', 'Minuscule 131')]
[('bookName', 'Anjou Legendarium')]
[('bookName', 'Codex Vaticanus 354')]
[('bookName', 'Codex Assemanius')]
[('bookName', 'Codex Ríos')]
[('bookName', 'Joshua Roll')]
[('bookName', 'breviary of  Matthias Corvin')]
[('bookName', 'Andrew, the Apostle')]
[('bookName', 'Apostolic Library Regina 1682')]
[('bookName', 'Apostolic Library Reg. 1323')]


20

In [67]:
#last stand, see if the author of a book stored in the vatican library brings us back to a nobel laureate

#found above in the long list of properties exiting a book
author = 'wdt:P50'

q = f"""
select distinct ?authorName ?book ?bookName where{{
?book wdt:P195|wdt:P276 wd:Q213678 ;
    <http://schema.org/name> ?bookName ;
    wdt:P50 ?author .
    
?author wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?authorName .
    
}}
limit 50
"""
run_query(q)

Empty


0

In [68]:
#let's remove as many constraints as possible

q = f"""
select distinct ?authorName ?book ?bookName ?property ?propertyName where{{
?book ?property wd:Q213678 ;
    <http://schema.org/name> ?bookName ;
    wdt:P50 ?author .
    
?property <http://schema.org/name> ?propertyName .
    
?author wdt:P166 wd:Q37922 ;
    <http://schema.org/name> ?authorName .
    
}}
limit 50
"""
run_query(q)

Empty


0

In [69]:
q = f"""
select distinct ?authorName ?book ?bookName ?property ?propertyName where{{
?book ?property wd:Q213678 ;
    <http://schema.org/name> ?bookName ;
    wdt:P50 ?author .
    
?property <http://schema.org/name> ?propertyName .
    
?author <http://schema.org/name> ?authorName .
    
}}
limit 50
"""
run_query(q)

[('authorName', 'Donizo of Canossa'), ('book', 'http://www.wikidata.org/entity/Q3561145'), ('bookName', 'Vita Mathildis'), ('property', 'http://www.wikidata.org/prop/direct/P195'), ('propertyName', 'collection')]
[('authorName', 'anonymous'), ('book', 'http://www.wikidata.org/entity/Q1106341'), ('bookName', 'Codex Vaticanus Graecus 64'), ('property', 'http://www.wikidata.org/prop/direct/P195'), ('propertyName', 'collection')]
[('authorName', 'Frederick II, Holy Roman Emperor'), ('book', 'http://www.wikidata.org/entity/Q1180580'), ('bookName', 'De arte venandi cum avibus'), ('property', 'http://www.wikidata.org/prop/direct/P195'), ('propertyName', 'collection')]
[('authorName', 'Frederick II, Holy Roman Emperor'), ('book', 'http://www.wikidata.org/entity/Q1180580'), ('bookName', 'De arte venandi cum avibus'), ('property', 'http://www.wikidata.org/prop/direct/P276'), ('propertyName', 'location')]
[('authorName', 'Leonardo da Vinci'), ('book', 'http://www.wikidata.org/entity/Q1258536'), (

15

In [70]:
#so there are authors with books stored on the vatican library, just no nobel laureates, at least according to wikidata

#let's make sure before quitting

q = f"""
select distinct ?author ?authorName ?book ?bookName where{{
?book ?property wd:Q213678 ;
    <http://schema.org/name> ?bookName ;
    wdt:P50 ?author .
    
?author <http://schema.org/name> ?authorName ;
    wdt:P166 wd:Q37922 .
    
}}
limit 50
"""
run_query(q)

Empty


0

In [72]:
#let's try the inverse

q = f"""
select distinct ?author ?authorName ?book ?bookName where{{

?author <http://schema.org/name> ?authorName ;
    wdt:P166 wd:Q37922 ;
    wdt:P800 ?book .
    
?book ?property wd:Q213678 ;
    <http://schema.org/name> ?bookName .
    
}}
limit 50
"""
run_query(q)

Empty


0